Hypothesis: It's really only worth using this for very large Hamiltonians, large enough that you can't precompute the fermion inner-products and hold them in memory. Otherwise, the benefit of precomputing the inner-products far outweighs anything you'll get from Numba/Cuda. 

In [1]:
import os 
import time
import copy
from tqdm import tqdm

import numpy as np 
import math
import matplotlib.pyplot as plt
import scipy
import pandas as pd

from joblib import Parallel, delayed
from numba import jit, njit, prange, cuda, float32, float64, complex64, complex128, types
import numba as nb

np.random.seed(0)

# Global variables
TPB = 16 # threads per block
BPG_MULTIPLIER = 1 # blocks per grid multiplier
REAL_FLOAT_TYPE = np.float64
COMPLEX_FLOAT_TYPE = np.complex128
REAL_INT_TYPE = np.int32
LOAD_PYTHON = False#True # Whether to generate Hamiltonians or load them from file (along with the associated coefficients, of course)

# Physical constants
K=10 # number of fermionic modes
J=4 # ~"energy scale"
Q=3 # order of coupling
N = 2*K # number of fermions
N_DIM = 2**K # Hilbert space dimensions

N_SAMPLES = 10 # number of samples to generate
N_JOBS = 20 # number of jobs to run in parallel

# 1. Define Python Hamiltonian as benchmark

In [2]:
#### Define fermionic modes
cr = np.array([[0,1],[0,0]])
an = np.array([[0,0],[1,0]])
id = np.identity(2)
id2 = np.array([[-1,0],[0,1]])

def c(n):
    factors = [id for i in range(n-1)]+[cr]+[id2 for i in range(K-n)]
    out = factors[0]
    for i in range(1, K):
        out = np.kron(out,factors[i])
    return out

def cd(n):
    factors = [id for i in range(n-1)]+[an]+[id2 for i in range(K-n)]
    out = factors[0]
    for i in range(1, K):
        out = np.kron(out,factors[i])
    return out

#### Define fermions
# Compute first N psi's
psi_h = np.zeros((N, N_DIM, N_DIM), dtype=COMPLEX_FLOAT_TYPE)
for i in range(1,K+1):
    psi_h[2*(i-1)] = (c(i)+cd(i))/np.sqrt(2)
    psi_h[2*(i-1)+1] = (c(i)-cd(i))*(-1j/np.sqrt(2))

## Copy to GPU
psi_d = cuda.to_device(psi_h)

def H3_python(js): #js being the random coefficients
    # Compute Hamiltonian
    H = np.zeros((N_DIM, N_DIM), dtype=COMPLEX_FLOAT_TYPE)
    for i in range(N-2):
        psi_i = psi_h[i]
        for j in range(i+1, N-1):
            psi_ij=psi_i@psi_h[j]
            for k in range(j+1, N):
                psi_ijk = psi_ij@psi_h[k]
                H += (1j**(Q/2))*js[i, j, k]*psi_ijk

    return H

#### Generate random coefficients
sigma_j = np.sqrt((J**2)*np.math.factorial(Q-1)/(N**(Q-1)))
js_all = [np.random.normal(0, sigma_j, size=tuple([N for i in range(Q)])) for j in range(N_SAMPLES+1)]

Don't run this one if you don't want to wait

In [3]:
# For K=7: 0.2 seconds
# For K=10: 62.3 SECONDS
if LOAD_PYTHON:
    js_test = np.load(os.path.join("Excel", "Benchmarks", f"js{Q}_benchmark.npy"))
    H3_python_test = np.load(os.path.join("Excel", "Benchmarks", f"H{Q}_python_benchmark.npy"))
else:                   
    js_test = js_all[0]

    tic = time.time()
    H3_python_test = H3_python(js_test)
    toc = time.time()
    duration = toc-tic
    print(f"Hamiltonian generation, python: {duration//60} minutes, {duration%60} seconds")

    np.save(os.path.join("Excel", "Benchmarks", f"js{Q}_benchmark.npy"), js_test)
    np.save(os.path.join("Excel", "Benchmarks", f"H{Q}_python_benchmark.npy"), H3_python_test)

Hamiltonian generation, python: 1.0 minutes, 12.107834815979004 seconds


# 2. Define hybrid Python-CUDA Hamiltonian

Method: Pretty much the same as the Python Hamiltonian, but use CUDA kernels to parallelize the matrix-multiplications and elementwise-addition.

$H = \sum_{k>j>i} J_{ijk} \psi_i \psi_j \psi_k$

$H[\alpha, \beta] = \sum_{k>j>i} J_{ijk} \left( \psi_i \psi_j \psi_k[\alpha, \beta] \right)$
$ = \sum_{k>j>i} J_{ijk} \left( \psi_{ijk}[\alpha, \beta] \right)$

Well, $\psi_l \psi_k [\alpha, \beta] = $

## Define CUDA fast matrix-multiply

In [4]:
# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
# TPB should not be larger than 32 in this example

@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B using CUDA shared memory.

    Reference: https://stackoverflow.com/a/64198479/13697228 by @RobertCrovella
    """
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=COMPLEX_FLOAT_TYPE)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=COMPLEX_FLOAT_TYPE)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = float32(0.)
    for i in range(bpg):
        # Preload data into shared memory
        sA[ty, tx] = 0
        sB[ty, tx] = 0
        if y < A.shape[0] and (tx + i * TPB) < A.shape[1]:
            sA[ty, tx] = A[y, tx + i * TPB]
        if x < B.shape[1] and (ty + i * TPB) < B.shape[0]:
            sB[ty, tx] = B[ty + i * TPB, x]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[ty, j] * sB[j, tx]

        # Wait until all threads finish computing
        cuda.syncthreads()
    if y < C.shape[0] and x < C.shape[1]:
        C[y, x] = tmp

x_h = np.arange(16).reshape([4, 4]).astype(COMPLEX_FLOAT_TYPE) + 1j*np.arange(16).reshape([4, 4]).astype(COMPLEX_FLOAT_TYPE)
y_h = np.ones([4, 4]).astype(COMPLEX_FLOAT_TYPE) +1j*np.ones([4, 4]).astype(COMPLEX_FLOAT_TYPE)
z_h = np.zeros([4, 4]).astype(COMPLEX_FLOAT_TYPE)

x_d = cuda.to_device(x_h)
y_d = cuda.to_device(y_h)
z_d = cuda.to_device(z_h)

threadsperblock = (TPB, TPB)
blockspergrid_x = math.ceil(BPG_MULTIPLIER*z_h.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(BPG_MULTIPLIER*z_h.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)

fast_matmul[blockspergrid, threadsperblock](x_d, y_d, z_d)
z_h = z_d.copy_to_host()
print(z_h)
print(x_h @ y_h)

c:\Users\abdel\anaconda3\envs\phys417_clone_07_10_23\lib\site-packages\numba\cuda\dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


[[0. +12.j 0. +12.j 0. +12.j 0. +12.j]
 [0. +44.j 0. +44.j 0. +44.j 0. +44.j]
 [0. +76.j 0. +76.j 0. +76.j 0. +76.j]
 [0.+108.j 0.+108.j 0.+108.j 0.+108.j]]
[[0. +12.j 0. +12.j 0. +12.j 0. +12.j]
 [0. +44.j 0. +44.j 0. +44.j 0. +44.j]
 [0. +76.j 0. +76.j 0. +76.j 0. +76.j]
 [0.+108.j 0.+108.j 0.+108.j 0.+108.j]]


In [5]:
@cuda.jit(types.void(float64[:,:], float64[:,:]))
def parallel_add(src, dest): # <-- THIS FUNCTION DOES NOT SUPPORT COMPLEX NUMBERS
    i, j = cuda.grid(2)
    if (i<dest.shape[0]) and (j<dest.shape[1]):
        cuda.atomic.add(dest, (i,j), src[i,j]) # <-- BECAUSE THIS FUNCTION DOES NOT SUPPORT COMPLEX NUMBERS

In [10]:
# For K=7: 0.5 seconds
# For K=10 with TPB=8, bpg_multiplier=1: 3 minutes, 13 seconds
# For K=10 with TPB=16, bpg_multiplier=1: 3 minutes, 13 seconds
# For K=10 with TPB=32, bpg_multiplier=1: 3 minutes 22 seconds... wtf?
# For K=10 with TPB=64, bpg_multiplier=1:
def H3_hybrid(H3_real_d, H3_imag_d, js_d):
    threadsperblock = (TPB, TPB)
    blockspergrid_x = math.ceil(N_DIM / threadsperblock[0])
    blockspergrid_y = math.ceil(N_DIM / threadsperblock[1])
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    for i in range(N-2):
        psi_i = psi_d[i]

        for j in range(i+1, N-1):
            psi_j = psi_d[j]
            psi_ij = cuda.device_array(shape=(N_DIM, N_DIM), dtype=COMPLEX_FLOAT_TYPE)
            fast_matmul[blockspergrid, threadsperblock](psi_i, psi_j, psi_ij)

            for k in range(j+1, N):
                psi_k = psi_d[k]
                psi_ijk = cuda.device_array(shape=(N_DIM, N_DIM), dtype=COMPLEX_FLOAT_TYPE)
                fast_matmul[blockspergrid, threadsperblock](psi_ij, psi_k, psi_ijk)

                j_ijk = js_d[i, j, k]
                addendum = (1j**(Q/2))*j_ijk*psi_ijk
                parallel_add[blockspergrid, threadsperblock](addendum, H3_real_d)
                parallel_add[blockspergrid, threadsperblock](1j*addendum, H3_imag_d)
    
    H3_real_h = H3_real_d.copy_to_host()
    H3_imag_h = H3_imag_d.copy_to_host()
    H3_h = H3_real_h - 1j*H3_imag_h
    return H3_h


# NOTE: cuda.device_array doesn't always work for some reason, doesn't update the defined device-array. Just keeps it as zeros. 
# For example, if you were to define 'H3_real_hybrid_test_d = cuda.device_array((N_DIM, N_DIM), dtype=np.complex128)' and then run 'H3_hybrid_test_h = H3_hybrid(H3_real_hybrid_test_d, H3_imag_hybrid_test_d, js_test_d)', H3_hybrid_test_h would just be an array of zeros at the end. 
H3_real_hybrid_test_h = np.zeros((N_DIM, N_DIM), REAL_FLOAT_TYPE)
H3_real_hybrid_test_d = cuda.to_device(H3_real_hybrid_test_h) 

H3_imag_hybrid_test_h = np.zeros((N_DIM, N_DIM), REAL_FLOAT_TYPE)
H3_imag_hybrid_test_d = cuda.to_device(H3_imag_hybrid_test_h) 

js_test_h = js_all[0].astype(COMPLEX_FLOAT_TYPE)
js_test_d = cuda.to_device(js_test_h)

tic = time.time()
H3_hybrid_test_h = H3_hybrid(H3_real_hybrid_test_d, H3_imag_hybrid_test_d, js_test_d)
toc = time.time()
duration = toc - tic
print(f"Duration: {duration//60} minutes, {duration%60} seconds")

c:\Users\abdel\anaconda3\envs\phys417_clone_07_10_23\lib\site-packages\numba\cuda\cudadrv\devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


Duration: 3.0 minutes, 58.22992444038391 seconds


NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.

  <-- This comes from 'psi' being called in the CUDA function, instead of 'psi_d'. Not sure how to copy list of scipy.sparse arrays to GPU yet, but can probably fix this in sparse-GPU implementation

K=7: 1.04 seconds

K=10: 205.3 seconds

In [11]:
print(np.allclose(H3_python_test, H3_hybrid_test_h))
print(np.allclose(H3_python_test.real, H3_hybrid_test_h.real))

True
True
